<a href="https://colab.research.google.com/github/AkankshaB123/leet-code/blob/main/New_Repeat_User(From_today's_date_7_days_window).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

- New: First-ever account <= 7 days old AND no historical payments.
- Repeat: Account >7 days old OR >= 1 historical payment.

In [1]:
# Install SQLite helper
import sqlite3
import pandas as pd
from datetime import datetime, timedelta

# ---------------------------
# 1. Create example data
# ---------------------------
today = datetime.today().date()

data = {
    "user_id": [1, 2, 3, 4, 5, 6],
    "signup_date": [
        today - timedelta(days=2),   # new, no txn
        today - timedelta(days=10),  # repeat, account old
        today - timedelta(days=3),   # new but has txn → repeat
        today - timedelta(days=20),  # repeat, old
        today - timedelta(days=1),   # new, no txn
        today - timedelta(days=5),   # new but has txn → repeat
    ],
    "txn_date": [
        None,
        None,
        today - timedelta(days=1),
        today - timedelta(days=5),
        None,
        today - timedelta(days=1),
    ]
}

df = pd.DataFrame(data)
df


,user_id,signup_date,txn_date
0,1,2025-12-07,None
1,2,2025-11-29,None
2,3,2025-12-06,2025-12-08
3,4,2025-11-19,2025-12-04
4,5,2025-12-08,None
5,6,2025-12-04,2025-12-08


In [2]:
# ---------------------------
# 2. Load data into SQLite
# ---------------------------
conn = sqlite3.connect(":memory:")
df.to_sql("txn_table", conn, index=False, if_exists="replace")


6

In [4]:
query = """
WITH user_txn AS (
    SELECT
        user_id,
        MIN(txn_date) AS first_txn_date,
        signup_date
    FROM txn_table
    GROUP BY user_id, signup_date
),
flagged AS (
    SELECT
        user_id,
        CASE
            WHEN signup_date > DATE('now', '-7 days')
                 AND first_txn_date IS NULL
                THEN 'new-user'
            ELSE 'repeat-user'
        END AS flag
    FROM user_txn
)
SELECT
    flag,
    COUNT(*) AS unique_users
FROM flagged
GROUP BY flag;
"""

result = pd.read_sql_query(query, conn)
result

,flag,unique_users
0,new-user,2
1,repeat-user,4
